## Rag (Method 1)

### Loading Model

In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('NL_January2024.pdf')
docs=loader.load()

### Text Splitting

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)

In [4]:
chunks = text_splitter.split_documents(docs)

In [5]:
chunks

[Document(metadata={'source': 'NL_January2024.pdf', 'page': 0}, page_content='DRDO QUALITY CONCLAVE 2023\nNEWSLETTER\nISSN: 0971-4391A Monthly Bulletin of Defence Research and Development Organisation\nJANUARY 2024 | VOLUME 44 | ISSUE 1\nwww.drdo.gov.in\nScan QR Code to access e-version of DRDO Newsletterhttps://www.drdo.gov.in/newsletter'),
 Document(metadata={'source': 'NL_January2024.pdf', 'page': 1}, page_content='2\n|   JANUARY 2024\nDRDO NEWSLETTER\nLABORATORY CORRESPONDENTSEditor-in-Chief: Dr K Nageswara Rao\nAssociate Editor-in-Chief: Sudhanshu BhushanEditor: Dipti AroraPre-press: Raj Kumar\n \nPrinting: SK Gupta  \nMarketing: Tapesh Sinha\nAhmadnagar  :  Shri RA Shaikh, Vehicle Research and Development Establishment (VRDE)\nAmbernath  : Dr Ganesh S Dhole, Naval Materials Research Laboratory (NMRL)\nChandipur  :  Shri PN Panda, Integrated Test Range (ITR) \n   Shri Ratnakar S, Mohapatra, P roof & Experimental Establishment (PXE) \nBengaluru  : Shri Satpal Singh Tomar, Aeronauti

### Embedding And Storing

In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embedding_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

c:\Users\Arjun\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Arjun\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
vector_db = Chroma.from_documents(documents=chunks,
                                  embedding=embedding_model,
                                  persist_directory="data_base_1",
                                  )


c:\Users\Arjun\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


##### Load VectorDB (if have)

In [8]:
vector_db = Chroma(persist_directory="data_base_1",
                   embedding_function=embedding_model)

c:\Users\Arjun\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(
c:\Users\Arjun\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [9]:
retriver = vector_db.as_retriever(top_k=3)

### Testing Retriever

In [13]:
vector_db.similarity_search("where run for unity organized ?", k=3)

[Document(metadata={'page': 8, 'source': 'NL_January2024.pdf'}, page_content="9\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nRUN FOR UNITY\nDL, Jodhpur\nOn the Rashtriya Ekta Diwas \norganised on 31 October 2023, \nall\tthe\tofficers\t and\tstaff\tof\tthe\t\nDefence Laboratory, Jodhpur (DLJ) took the Rashtriya Ekta Diwas Pledge. Further, ‘Run for Unity’ was organised to spread the message of unity in society. The event was led by Shri RV Hara \nPrasad, Outstanding Scientist & \nDirector, DLJ. During his address, he highlighted the contributions made by the legend Sardar Vallabh Bhai Patel in the freedom struggle \nand national integration after \nindependence.\nISSA, Delhi \nOn 31 October 2023, Institute \nfor Systems Studies & Analyses \n(ISSA), Delhi, organised ‘Run for Unity’ from Metcalfe House to Qudasia Bagh, near ISBT \nKashmere Gate. It is an 18th-\ncentury garden complex with depictions of the Indian freedom struggle movement. \nAll\tofficers\t and\tstaff\tof\tthe\t\nlaboratory t

### Prompt Passing

In [10]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")


In [11]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """ 
    Answer the following question based on the provided context.
    Note : analyse given context carefully
    <context>
    {context}
    </context>

    Question : {input}""")


In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
from langchain.chains import create_retrieval_chain
retrival_chain = create_retrieval_chain(retriver, document_chain)

In [14]:
ans = retrival_chain.invoke({"input" : "List of all the places where RUN FOR UNITY was organized"})

In [15]:
ans

{'input': 'List of all the places where RUN FOR UNITY was organized',
 'context': [Document(metadata={'page': 8, 'source': 'NL_January2024.pdf'}, page_content="9\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nRUN FOR UNITY\nDL, Jodhpur\nOn the Rashtriya Ekta Diwas \norganised on 31 October 2023, \nall\tthe\tofficers\t and\tstaff\tof\tthe\t\nDefence Laboratory, Jodhpur (DLJ) took the Rashtriya Ekta Diwas Pledge. Further, ‘Run for Unity’ was organised to spread the message of unity in society. The event was led by Shri RV Hara \nPrasad, Outstanding Scientist & \nDirector, DLJ. During his address, he highlighted the contributions made by the legend Sardar Vallabh Bhai Patel in the freedom struggle \nand national integration after \nindependence.\nISSA, Delhi \nOn 31 October 2023, Institute \nfor Systems Studies & Analyses \n(ISSA), Delhi, organised ‘Run for Unity’ from Metcalfe House to Qudasia Bagh, near ISBT \nKashmere Gate. It is an 18th-\ncentury garden complex with depictions of the India

In [16]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

'Based on the provided context, the places where "RUN FOR UNITY" was organized are:    1. DL, Jodhpur (Defence Laboratory, Jodhpur)  2. ISSA, Delhi (Institute for Systems Studies & Analyses, Delhi)  3. MTRDC, Bengaluru (Microwave Tube Research & Development Centre, Bengaluru)  4. NSTL, Visakhapatnam (Naval Science & Technological Laboratory, Visakhapatnam)'

In [17]:
ans2 = retrival_chain.invoke({"input" : "List of all the places where Vigilance Awareness Week was organized"})

In [18]:
ans2

{'input': 'List of all the places where Vigilance Awareness Week was organized',
 'context': [Document(metadata={'page': 7, 'source': 'NL_January2024.pdf'}, page_content="8\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nVIGILANCE AWARENESS WEEK\nVigilance Awareness Week is observed every year during the week in which the birthday of \nSardar Vallabhbhai Patel (31 October) falls. The theme of this year’s Vigilance Awareness Week was ‘Say No to Corruption, Commit to the Nation’.\nDL, Jodhpur\nVigilance Awareness Week was \ncelebrated during 30 October to 3 November 2023 at the Defence \nLaboratory, Jodhpur (DLJ). All \nthe\tofficers\tand\tstaff\ttook\ta\tpledge\t\nregarding vigilance. A lecture on 'Say No to Corruption: Commit \nto the Nation' was delivered by \nShri Harendra Kumar Mahawar, \nIPS, IG, Anti-Corruption Bureau, \nJodhpur, on the occasion.\nIRDE, Dehradun\nInstruments Research & \nDevelopment Establishment (IRDE), Dehradun, observed \nVigilance Awareness Week during \n30 October 

In [19]:
st2 = ans2['answer']
stt2 = st2.replace("\n", "  ")
stt2

'Based on the provided context, there is no mention of Vigilance Awareness Week. The events mentioned are "Run for Unity" celebrations held by various laboratories and research centers on Rashtriya Ekta Diwas (National Unity Day) on October 31, 2023.    However, if you\'re looking for a list of places where the "Run for Unity" event was organized, here it is:    1. ISSA, Delhi  2. MTRDC, Bengaluru  3. NSTL, Visakhapatnam  4. DLJ, Jodhpur'

In [20]:
ans = retrival_chain.invoke({"input" : "Give me the summary of KARNATAKA RAJYOTSAVA CELEBRATION"})

In [21]:
ans

{'input': 'Give me the summary of KARNATAKA RAJYOTSAVA CELEBRATION',
 'context': [Document(metadata={'page': 10, 'source': 'NL_January2024.pdf'}, page_content='11\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\n(NPOL), Kochi celebrated its \n71st Annual Day on 15 December 2023. Dr S Unnikrishnan Nair, DS \n& Director, VSSC was the Chief \nGuest for the event. Dr R Rajesh, Sc ‘G’ & Chairman, NPOL Annual Day Organising Committee welcomed the gathering and presented an overview of the \nfunction. Dr Ajith Kumar K, OS \n& Director, NPOL presented the report on the achievements of the laboratory in both technical and non-technical areas. Shri \nWilson K Cherukulath, Sc ‘G’ & \nChairman, Works Committee & Secretary, Works Committee and General Secretaries of NPOL’s Civilian Employees union and NPOL’s R&D Employees union \nalso spoke on the occasion. Prizes \nwere distributed for the sports competitions held during the Annual Day. Instrumental music fusion by ‘RLV College of Music \nand Fine Arts’ 

In [22]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

"Based on the provided context, here is a summary of the KARNATAKA RAJYOTSAVA CELEBRATION:    The Microwave Tube Research & Development Centre (MTRDC), Bengaluru, celebrated Karnataka Rajyotsava, the State Formation Day, on November 17, 2023. The event was presided over by Dr. Ganesha Bhatt, Retired Principal of MES educational institution, who spoke about Kannada, its rich heritage, and the language's high number of Jnanapith awards.    The celebration included a cultural program, where Shri Srinidhi Katti, a professional flute maestro, mesmerized the audience with his flute performance. The event also featured addresses by Dr. SK Datta, Centre Head, MTRDC, and Shri G Viswam, Director, LRDE & MTRDC.    Overall, the celebration aimed to commemorate Karnataka's State Formation Day and promote cultural appreciation for the Kannada language."

In [23]:
ans = retrival_chain.invoke({"input" : "List of all the places where SWACHHATA PAKHWADA was organized"})

In [24]:
ans

{'input': 'List of all the places where SWACHHATA PAKHWADA was organized',
 'context': [Document(metadata={'page': 13, 'source': 'NL_January2024.pdf'}, page_content='14\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nSWACHHATA PAKHWADAwas invited as a guest speaker on \n05 Decemebr 2023. She delivered a lecture on ‘Importance of Millets’ \nand\tfocused\t on\tthe\tbenefits\t of\t\nmillets in the diet. Shri PT Rojatkar, \nOS & Director, NMRL felicitated the speaker. Cooking and poster competitions on millets were also organised on 11 December \n2023. NMRL employees and their \nfamily members took part in the competition. Over 200 people visited the exhibition where the posters were displayed. The \nwinners were awarded with the \nprices\tand\tcertificates.\nDESIDOC, Delhi\nThe\t Defence\t Scientific\t\nInformation & Documentation Centre (DESIDOC), Delhi, \norganised a swachhata Pakhwada \nduring 01-15 December 2023. Besides the cleanliness drive during the two-week-long program, a couple of ev

In [25]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

'Based on the provided context, the following places where Swachhata Pakhwada was organized are:    1. SFC (SF Complex), Jagdalpur  2. Defence Laboratory School, Kanchanbagh  3. Kendriya Vidyalaya, Kanchanbagh  4. IRDE (Instruments Research & Development Establishment), Dehradun'